In [ ]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from collections import defaultdict
from PIL import Image
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Définir les chemins vers les dossiers d'entraînement et de validation
train_dir = 'C:\\Users\\ITLAB-CLIENT05\\Desktop\\Sarah\\datasignature\\entrainement'
val_dir = 'C:\\Users\\ITLAB-CLIENT05\\Desktop\\Sarah\\datasignature\\validation'

In [ ]:
# Paramètres
img_height, img_width = 128, 128  # Dimensions des images
batch_size = 32  # Taille du batch
num_classes = 114  # Nombre de classes

In [ ]:
# Générateurs de données
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Création des générateurs de données pour les vraies signatures uniquement
train_generator = train_datagen.flow_from_directory(
    os.path.join(train_dir, 'Vraies'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(val_dir, 'Vraies'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'
)

In [ ]:
# Générateur de paires ancre-positive
class AnchorPositivePairs(tf.keras.utils.Sequence):
    def __init__(self, generator, num_batches):
        self.generator = generator
        self.num_batches_ap = num_batches
        self.class_idx_to_train_idxs = defaultdict(list)
        for i in range(len(generator)):
            _, labels = generator[i]
            for idx, label in enumerate(labels):
                self.class_idx_to_train_idxs[int(label)].append((i, idx))
    
    def __len__(self):
        return self.num_batches_ap

    def __getitem__(self, _idx):
        anchors = np.empty((num_classes, img_height, img_width, 3), dtype=np.float32)
        positives = np.empty((num_classes, img_height, img_width, 3), dtype=np.float32)
        for class_idx in range(num_classes):
            examples_for_class = self.class_idx_to_train_idxs[class_idx]
            anchor_idx, anchor_sub_idx = random.choice(examples_for_class)
            positive_idx, positive_sub_idx = random.choice(examples_for_class)
            while positive_idx == anchor_idx and positive_sub_idx == anchor_sub_idx:
                positive_idx, positive_sub_idx = random.choice(examples_for_class)
            anchors[class_idx] = self.generator[anchor_idx][0][anchor_sub_idx]
            positives[class_idx] = self.generator[positive_idx][0][positive_sub_idx]
        return anchors, positives

In [ ]:
# Modèle d'apprentissage
class EmbeddingModel(tf.keras.Model):
    def train_step(self, data):
        anchors, positives = data

        with tf.GradientTape() as tape:
            anchor_embeddings = self(anchors, training=True)
            positive_embeddings = self(positives, training=True)

            similarities = tf.einsum("ae,pe->ap", anchor_embeddings, positive_embeddings)
            temperature = 0.2
            similarities /= temperature

            sparse_labels = tf.range(num_classes)
            loss = self.compiled_loss(sparse_labels, similarities)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        self.compiled_metrics.update_state(sparse_labels, similarities)
        return {m.name: m.result() for m in self.metrics}

inputs = layers.Input(shape=(img_height, img_width, 3))
x = layers.Conv2D(filters=32, kernel_size=3, strides=2, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, strides=2, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, strides=2, activation="relu")(x)
x = layers.GlobalAveragePooling2D()(x)
embeddings = layers.Dense(units=512, activation=None)(x)
embeddings = tf.keras.layers.LayerNormalization()(embeddings)

model = EmbeddingModel(inputs, embeddings)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

In [ ]:
# Entraîner le modèle
train_data = AnchorPositivePairs(train_generator, num_batches=1000)
history = model.fit(train_data, epochs=2)

In [ ]:
# Afficher la courbe de perte
plt.plot(history.history["loss"])
plt.title("Courbe de perte")
plt.xlabel("Époque")
plt.ylabel("Perte")
plt.show()

In [ ]:
# Évaluer le modèle
val_data = AnchorPositivePairs(val_generator, num_batches=len(val_generator))
embeddings = model.predict(val_data)
gram_matrix = np.einsum("ae,be->ab", embeddings, embeddings)
near_neighbours = np.argsort(gram_matrix.T)[:, -(10 + 1) :]

In [ ]:
# Afficher la matrice de confusion
confusion_matrix = np.zeros((num_classes, num_classes))
for class_idx in range(num_classes):
    example_idxs = val_generator.index_array[val_generator.classes == class_idx][:10]
    for y_test_idx in example_idxs:
        for nn_idx in near_neighbours[y_test_idx][:-1]:
            nn_class_idx = val_generator.classes[nn_idx]
            confusion_matrix[class_idx, nn_class_idx] += 1

labels = ["Classe {}".format(i) for i in range(num_classes)]
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=labels)
disp.plot(include_values=True, cmap="viridis", ax=None, xticks_rotation="vertical")
plt.show()